In [1]:
import findspark
findspark.init('/usr/local/spark')

In [2]:
from pyspark import SparkContext, SparkConf, SQLContext
import pandas as pd
import re
from operator import add
import numpy as np
import hashlib
from datasketch.minhash import MinHash
from datasketch.lsh import MinHashLSH
import csv

In [3]:
sc = SparkContext.getOrCreate()
#questionFile=sc.textFile("Questions.csv")
#answerFile=sc.textFile("Answers.csv")
questionFile=sc.textFile("test_q.csv")
answerFile=sc.textFile("test_a.csv")
tagFile = sc.textFile("test_t.csv")
stopWordsFile = sc.textFile("stopwords.txt")
print("finished")

finished


In [4]:
#Read sample of question.csv(5%), generate the keywords list to focus on

In [5]:
questionWithHeader = questionFile.map(lambda line: line.split(",")).filter(lambda line: len(line)>1)
header = questionWithHeader.first() #extract header
question = questionWithHeader.filter(lambda x: x != header)
print(question.count())

9999


In [6]:
questionSample=question.sample(False,0.05,81)
questionLower = questionSample.map(lambda x: (x[0], re.sub("<.*?>", " ", (x[5]+x[6]).lower())))
def f(x): return re.sub('[^a-zA-Z0-9\']+', ' ', x)
questionLower = questionLower.mapValues(f)
def f2(x): return x.split(" ")
questionPairRaw = questionLower.flatMapValues(f2)
print(questionPairRaw.count())

62192


In [7]:
stopWordsList = stopWordsFile.collect()
questionPair=questionPairRaw.filter(lambda x: x[1] not in stopWordsList)
questionPairfilter=questionPair.filter(lambda x:x[1]!="")
questionPairStage1 = questionPairfilter.map(lambda x:(x[0]+"@"+x[1],1)).reduceByKey(add)
N = len(questionPair.countByKey())
print(N)

491


In [8]:
questionPairStage2MapPhase = questionPairStage1.map(lambda x:(x[0].split("@")[1],x[0].split("@")[0]+"="+str(x[1])))
stage2Map = questionPairStage2MapPhase.countByKey()
questionPairStage2 = questionPairStage2MapPhase.map(lambda x:(x[0]+"@"+x[1].split("=")[0],(1+np.log(int(x[1].split("=")[1])))*np.log(N/stage2Map.get(x[0]))))
questionPairStage3MapPhase = questionPairStage2.map(lambda x:(x[0].split("@")[1],x[0].split("@")[0]+"="+str(x[1])))
def f3_1(a,b): return float(a)+float(b.split("=")[1])*float(b.split("=")[1])
def f3_2(a,b): return float(a)+float(b)
questionPairStage3AggByKey = questionPairStage3MapPhase.aggregateByKey(0.0,f3_1,f3_2)
print(questionPairStage3AggByKey.count())

491


In [9]:
questionPairStage3AggByKeysqr = questionPairStage3AggByKey.map(lambda x:(x[0],np.sqrt(x[1])))
stage3Map = questionPairStage3AggByKeysqr.collectAsMap()
questionPairStage3 = questionPairStage3MapPhase.map(lambda x:(x[0],x[1].split("=")[0]+"="+str(float(x[1].split("=")[1])/stage3Map.get(x[0]))))
num_keywords = 7
def f4_1(a,b):
    c=[]
    for i in range(num_keywords):
        if float(b.split("=")[1])>float(a[i].split("=")[1]):
            a[i]=b
            break
    return a
def f4_2(a,b):
    for i in range(num_keywords): #b
        for j in range(num_keywords): #a
            if float(b[i].split("=")[1])>float(a[j].split("=")[1]):
                a[j] = b[i]
                break
    return a
questionPairStage4 = questionPairStage3.aggregateByKey(["a=0","b=0","c=0","d=0","e=0","f=0","g=0"],f4_1,f4_2)
print(questionPairStage4.count())
questionPairStage4.take(5)

491


[('4320',
  ['bll=0.37277064711218366',
   'logic=0.2759487925563063',
   'procs=0.17762721066918158',
   'validatable=0.17762721066918158',
   "presentation's=0.17762721066918158",
   'objectdatasource=0.17762721066918158',
   'preferable=0.17762721066918158']),
 ('16970',
  ['notes=0.40068292564436403',
   'offers=0.40068292564436403',
   'docs=0.2966112551673511',
   'perfect=0.2101776507655334',
   'decent=0.2101776507655334',
   'clients=0.16233309863451986',
   'existing=0.1572333712030617']),
 ('17020',
  ['partitions=0.3632129696362643',
   'terabyte=0.21451943091925724',
   '2gb=0.21451943091925724',
   '2x=0.21451943091925724',
   'ram=0.21451943091925724',
   'tomorrow=0.21451943091925724',
   'divide=0.21451943091925724']),
 ('20840',
  ['dirty=0.46364917386085336',
   'reads=0.43748441216684975',
   'isolation=0.33285249080393287',
   'uncommitted=0.33285249080393287',
   'read=0.14503166535203965',
   'f=0',
   'g=0']),
 ('22340',
  ['wcf=0.2690669399286497',
   'firewall

In [10]:
def fmap1(x):
    output=""
    for i in range(num_keywords):
        y=x[1][i].split("=")
        if float(y[1])>0.4:
           output+=y[0]+";"
    return ("key",output)
keywordlist=questionPairStage4.map(fmap1).reduceByKey(add)
keywords=keywordlist.take(1)
words=keywords[0][1][:-1]
keywordset=set(words.split(";"))
print(len(keywordset))
print(keywordset)

256
{'connectionstring', 'cooliris', 'offers', 'reports', 'vb', 'processes', 'fgets', 'differences', 'uiwebview', 'binding', 'ray', 'timer', 'ajax', 'cronjob', 'convert', 'alternate', '2000', 'mdi', 'youtube', '400', 'mvc', 'campaign', 'trigger', 'ntfs', 'gpl', 'burn', 'resin', 'axhost', 'oracle', 'character', 'certificate', 'vista', 'concatenate', 'foreign', 'val2', 'xsl', 'url', 'watermark', 'configurations', 'slave', 'arabic', 'menu', 'hour', 'grep', 'ratio', 'growth', 'setout', 'facility', 'httpclient', 'deploying', 'playerwin', 'llvm', 'notes', 'integers', 'vision', 'session', 'orm', 'asynchronous', 'wikipedia', 'simulate', 'newest', '9241', 'berkley', 'front', 'workflow', 'mac', 'varchar', 'factors', 'credit', 'formats', 'practices', 'backbutton', 'restful', 'mm', 'myobj', 'cluster', 'viewdata', 'datagrid', 'abc', 'dirty', 'overused', 'unhandledexceptionmode', 'thumbnail', 'window', 'patterns', 'gdb', 'table1', 'cvs', 'javadoc', 'clojure', 'jtracert', 'addins', 'sprocs', 'precedi

In [11]:
#turn oringinal question pairs to pairs only keep the keywords,get 7 keywords for each question

In [12]:
questionLower = question.map(lambda x: (x[0], re.sub("<.*?>", " ", (x[5]+x[6]).lower())))
def f(x): return re.sub('[^a-zA-Z0-9\']+', ' ', x)
questionLower = questionLower.mapValues(f)
def f2(x): return x.split(" ")
questionPairRaw = questionLower.flatMapValues(f2)
questionPair=questionPairRaw.filter(lambda x: x[1] in keywordset)
print(questionPair.count())

25302


In [13]:
questionPairfilter=questionPair.filter(lambda x:x[1]!="")
questionPairStage1 = questionPairfilter.map(lambda x:(x[0]+"@"+x[1],1)).reduceByKey(add)
N = len(questionPair.countByKey())
questionPairStage2MapPhase = questionPairStage1.map(lambda x:(x[0].split("@")[1],x[0].split("@")[0]+"="+str(x[1])))
stage2Map = questionPairStage2MapPhase.countByKey()
questionPairStage2 = questionPairStage2MapPhase.map(lambda x:(x[0]+"@"+x[1].split("=")[0],(1+np.log(int(x[1].split("=")[1])))*np.log(N/stage2Map.get(x[0]))))
print(questionPairStage2.count())
print(questionPairStage2.take(5))

12812
[('varchar@80', 11.98072901512991), ('menu@120', 4.071822723979202), ('controls@120', 3.374374461519866), ('card@260', 13.827548207665414), ('language@260', 5.043841666149129)]


In [14]:
questionPairStage3MapPhase = questionPairStage2.map(lambda x:(x[0].split("@")[1],x[0].split("@")[0]+"="+str(x[1])))
def f3_1(a,b): return float(a)+float(b.split("=")[1])*float(b.split("=")[1])
def f3_2(a,b): return float(a)+float(b)
questionPairStage3AggByKey = questionPairStage3MapPhase.aggregateByKey(0.0,f3_1,f3_2)
questionPairStage3AggByKeysqr = questionPairStage3AggByKey.map(lambda x:(x[0],np.sqrt(x[1])))
stage3Map = questionPairStage3AggByKeysqr.collectAsMap()
print(stage3Map.get('90'))

None


In [15]:
questionPairStage3 = questionPairStage3MapPhase.map(lambda x:(x[0],x[1].split("=")[0]+"="+str(float(x[1].split("=")[1])/stage3Map.get(x[0]))))

num_keywords = 7
def f4_1(a,b):
    c=[]
    for i in range(num_keywords):
        if float(b.split("=")[1])>float(a[i].split("=")[1]):
            a[i]=b
            break
    return a
def f4_2(a,b):
    for i in range(num_keywords): #b
        for j in range(num_keywords): #a
            if float(b[i].split("=")[1])>float(a[j].split("=")[1]):
                a[j] = b[i]
                break
    return a
questionPairStage4 = questionPairStage3.aggregateByKey(["a=0","b=0","c=0","d=0","e=0","f=0","g=0"],f4_1,f4_2)
print(questionPairStage4.take(5))

[('330', ['benefits=0.858597929783121', 'b=0', 'c=0', 'd=0', 'e=0', 'f=0', 'g=0']), ('580', ['deploying=0.792509095806917', 'test=0.6098600930240499', 'c=0', 'd=0', 'e=0', 'f=0', 'g=0']), ('2530', ['tag=1.0', 'b=0', 'c=0', 'd=0', 'e=0', 'f=0', 'g=0']), ('2900', ['logging=0.7658192589148438', 'rows=0.6430558783458242', 'c=0', 'd=0', 'e=0', 'f=0', 'g=0']), ('2970', ['practices=0.6558321069911532', 'url=0.47557508058352693', 'c=0', 'd=0', 'e=0', 'f=0', 'g=0'])]


In [16]:
outMap=questionPairStage4.collectAsMap()
tagWithHeader = tagFile.map(lambda line: line.split(",")).filter(lambda line: len(line)>1)
header = tagWithHeader.first() #extract header
tag = tagWithHeader.filter(lambda x: x != header)
tagLower = tag.map(lambda x: (x[0], x[1].lower()))
def f0(x): return re.sub('[^a-zA-Z]+', '', x)
tagLower = tagLower.mapValues(f0)
tag_1000 = tagLower.map(lambda x: (x[0], x[1]+"=1000"))
questionPairStage5=questionPairStage4.map(lambda x:(x[0],x[1][0]+":"+x[1][1]+":"+x[1][2]+":"+x[1][3]+":"+x[1][4]+":"+x[1][5]+":"+x[1][6]) )
print(questionPairStage5.take(5))

[('330', 'benefits=0.858597929783121:b=0:c=0:d=0:e=0:f=0:g=0'), ('580', 'deploying=0.792509095806917:test=0.6098600930240499:c=0:d=0:e=0:f=0:g=0'), ('2530', 'tag=1.0:b=0:c=0:d=0:e=0:f=0:g=0'), ('2900', 'logging=0.7658192589148438:rows=0.6430558783458242:c=0:d=0:e=0:f=0:g=0'), ('2970', 'practices=0.6558321069911532:url=0.47557508058352693:c=0:d=0:e=0:f=0:g=0')]


In [17]:
questionPairStage6=questionPairStage5.union(tag_1000)
def f6_1(a,b):
    return a+":"+b
def f6_2(a,b):
    return a+b
questionPairStage6=questionPairStage6.aggregateByKey("",f6_1,f6_2).map(lambda x: (x[0], x[1][1:]))

In [18]:
print(questionPairStage6.take(10))

[('330', 'benefits=0.858597929783121:b=0:c=0:d=0:e=0:f=0:g=0:c=1000:oop=1000:class=1000:nestedclass=1000'), ('580', 'deploying=0.792509095806917:test=0.6098600930240499:c=0:d=0:e=0:f=0:g=0:sqlserver=1000:sqlserver=1000:deployment=1000:releasemanagement=1000'), ('2970', 'practices=0.6558321069911532:url=0.47557508058352693:c=0:d=0:e=0:f=0:g=0:security=1000:cracking=1000:hijacked=1000'), ('3790', '64=0.7504686006209328:vista=0.3974409339025996:c=0:d=0:e=0:f=0:g=0:windows=1000:bit=1000:wmi=1000'), ('5460', 'tag=1.0:b=0:c=0:d=0:e=0:f=0:g=0:communityserver=1000'), ('5690', 'mvc=0.7973741074052297:convert=0.45246774118980443:c=0:d=0:e=0:f=0:g=0:aspnetmvc=1000:aspnetmvcrouting=1000'), ('6130', 'window=0.8145084617346454:b=0:c=0:d=0:e=0:f=0:g=0:svn=1000:subclipse=1000'), ('7990', 'stringtemplate=0.8878681890649611:class=0.19074254500279683:c=0:d=0:e=0:f=0:g=0:c=1000:net=1000:windowsservices=1000:printing=1000'), ('10670', 'singleton=0.7044793097864777:processing=0.2987256764556073:c=0:d=0:e=0:

In [19]:
questionKeywordAndTag = questionPairStage6.flatMapValues(lambda x: x.split(":")).filter(lambda x: float(x[1].split("=")[1]) > 0)
questionKeywordAndTag = questionKeywordAndTag.map(lambda x: (x[0], (x[1].split("=")[0], x[1].split("=")[1])))
print(questionKeywordAndTag.take(5))

questionKeywordAndTagSet = set(questionKeywordAndTag.map(lambda x: x[1][0]).collect())
print(len(questionKeywordAndTagSet))

[('330', ('benefits', '0.858597929783121')), ('330', ('c', '1000')), ('330', ('oop', '1000')), ('330', ('class', '1000')), ('330', ('nestedclass', '1000'))]


In [20]:
partitionNum = 100
def hashKeyword(x):
    return (int(hashlib.sha1(x.encode()).hexdigest(), 16) % (10 ** 8)) % partitionNum

questionHashedKeywordAndTag = questionKeywordAndTag.map(lambda x: (x[0], (hashKeyword(x[1][0]), x[1][1])))
print(questionHashedKeywordAndTag.take(5))

[('330', (45, '0.858597929783121')), ('330', (36, '1000')), ('330', (71, '1000')), ('330', (46, '1000')), ('330', (23, '1000'))]


In [21]:
questionPartitions = questionHashedKeywordAndTag.map(lambda x: (x[1][0], x[0]))
questionPartitions = questionPartitions.groupByKey().sortBy(lambda x: x[0]).map(lambda x: (x[0], set(x[1])))
print(questionPartitions.count())

100


In [22]:
newQuestion = sc.wholeTextFiles("new_q.txt")
newQuestion = newQuestion.map(lambda x: re.sub('\n', ' ', x[1]))
newQuestion = newQuestion.map(lambda x: re.sub('<.*?>', ' ', x.lower()))
newQuestion = newQuestion.map(lambda x: re.sub('[^a-zA-Z0-9\']+', ' ', x))

newQuestionKeywordsStageOne = newQuestion.flatMap(lambda x: x.lower().split(" ")).filter(lambda x: x != '' and x in questionKeywordAndTagSet).map(lambda x: (x, 1)).reduceByKey(add).map(lambda x: (x[0], 'new=' + str(x[1])))
print(newQuestionKeywordsStageOne.count())

newQuestionKeywordsStageTwoMap = newQuestionKeywordsStageOne.union(questionPairStage2MapPhase).countByKey()
newQuestionKeywordsStageTwo = newQuestionKeywordsStageOne.map(lambda x: (x[0], (1 + np.log(int(x[1].split("=")[1]))) * np.log((N + 1)/newQuestionKeywordsStageTwoMap.get(x[0]))))
print(newQuestionKeywordsStageTwo.count())

newQuestionKeywordsStageThreeSquare = newQuestionKeywordsStageTwo.map(lambda x: np.square(x[1])).reduce(add)
newQuestionKeywordsStageThree = newQuestionKeywordsStageTwo.map(lambda x: (x[0], x[1]/np.sqrt(newQuestionKeywordsStageThreeSquare)))
print(newQuestionKeywordsStageThree.take(5))

keywordAndTagSet = keywordset.union(tagLower.values().collect())
newQuestionKeywordsStageFour = newQuestionKeywordsStageThree.filter(lambda x: x[0] in keywordAndTagSet).sortBy(lambda x: -x[1])
print(newQuestionKeywordsStageFour.take(5))

17
17
[("i'm", 0.2103504541639918), ('spark', 0.35615427839726677), ('order', 0.2103504541639918), ('answers', 0.2103504541639918), ('java', 0.2103504541639918)]
[('java', 0.2103504541639918), ('dataset', 0.2103504541639918)]


In [23]:
keywordNum = 10
newQuestionKeywords = newQuestionKeywordsStageFour.zipWithIndex().filter(lambda x: x[1] < keywordNum).keys()
print(newQuestionKeywords.take(keywordNum))

[('java', 0.2103504541639918), ('dataset', 0.2103504541639918)]


In [24]:
newQuestionHashedKeywords = newQuestionKeywords.map(lambda x: hashKeyword(x[0])).collect()
print(newQuestionHashedKeywords)
bucketQuestions = questionPartitions.filter(lambda x: x[0] in newQuestionHashedKeywords).map(lambda x: x[1]).reduce(lambda a, b: a.union(b))
print(len(bucketQuestions))

[72, 54]
532


In [25]:
# union new question to similar question
candidateSimilarQuestions = questionLower.filter(lambda x: x[0] in bucketQuestions)
candidateSimilarQuestions = newQuestion.map(lambda x: ['0', x]).union(candidateSimilarQuestions)
candidateSimilarQuestions = candidateSimilarQuestions.map(lambda x: (x[0], list(filter(None, x[1].split(" ")))))
print(candidateSimilarQuestions.count())

# generate shingles
shingleLength = 2
candidateQuestionShingles = candidateSimilarQuestions.map(lambda x: (x[0], [" ".join(x[1][i: i + shingleLength]) for i in range(len(x[1]) - shingleLength + 1)]))
print(candidateQuestionShingles.take(1))

# hash shingles
candidateQuestionHashedShingles = candidateQuestionShingles.map(lambda x: (x[0], [hashlib.sha1(x[1][i].encode()).hexdigest() for i in range(len(x[1]) - 1)]))
print(candidateQuestionHashedShingles.take(1))

# minhash
numPerm = 128
m = {}
def minhash(x):
    m[x[1]] = MinHash(num_perm = numPerm)
    for shingle in x[0][1]:
        m[x[1]].update(shingle.encode('utf8'))
    return (x[0][0], m[x[1]])
candidateQuestionMinhash = candidateQuestionHashedShingles.zipWithIndex().map(minhash)

newQuestionMinhash = candidateQuestionMinhash.take(1)[0][1]
candidateQuestionMinhash = candidateQuestionMinhash.filter(lambda x: x[0] != '0')

# lsh
threshold = 0.1
lsh = MinHashLSH(threshold = threshold, num_perm = numPerm)
with lsh.insertion_session() as session:
    for x in candidateQuestionMinhash.collect():
        session.insert(x[0], x[1])
similarQuestions = lsh.query(newQuestionMinhash)
print(len(similarQuestions))

533
[('0', ["i'm using", 'using spark', 'spark in', 'in order', 'order to', 'to suggest', 'suggest some', 'some answers', 'answers but', 'but i', 'i keep', 'keep getting', 'getting spark', 'spark java', 'java lang', 'lang stackoverflowerror', 'stackoverflowerror when', 'when i', 'i run', 'run my', 'my code', 'code on', 'on a', 'a big', 'big dataset', 'dataset 40k', '40k entries', 'entries when', 'when running', 'running the', 'the code', 'code on', 'on a', 'a small', 'small number', 'number of', 'of entries', 'entries it', 'it works', 'works fine', 'fine though'])]
[('0', ['99a3da0112d45b4873e0ae7ce422888d45fa14c8', '129275a321e591df298ff9f03bf9bb6de9088862', 'fee51adbdffb4e8688b46fee7a2769cf8d17e306', '33f6a18ad70cf57665ed9259401237cbff72ec37', '8bed31a162636069e651f82f882b796600f49302', '124cbe7bd236e0cce71ffda5979cc4a0a6d74a62', '2dd5c133a748d53b4e60d995f2f41afb7d4397f1', 'c00ff05b5c1c2aaac10a3668d4493e45db23a0af', 'b028498210c5464e25111ee73315eb73d1568ade', '683c79e433c6646d12e9f1e

In [26]:
sqlContext = SQLContext(sc)

questionDataFrame = sqlContext.read.format("csv").options(header="true").load("Questions.csv")
answerDataFrame = sqlContext.read.format("csv").options(header="true").load("Answers.csv")
tagDataFrame = sqlContext.read.format("csv").options(header="true").load("Tags.csv")

print("finish reading source files")

finish reading source files


In [27]:
similarQuestionMinhash = candidateQuestionMinhash.filter(lambda x: x[0] in similarQuestions)
similarQuestionJaccardSimilarity = similarQuestionMinhash.map(lambda x: (x[0], newQuestionMinhash.jaccard(x[1]))).collectAsMap()

similarQuestionDataFrame = questionDataFrame.filter(questionDataFrame.Id.isin(similarQuestions))
similarQuestionDataFrame = similarQuestionDataFrame.join(answerDataFrame, questionDataFrame.Id == answerDataFrame.ParentId).select(questionDataFrame.Id, questionDataFrame.Body, answerDataFrame.Body, answerDataFrame.Score)
print(similarQuestionDataFrame.take(1))

similarQuestionAnswers = similarQuestionDataFrame.rdd.map(lambda x: (x[0], ((x[1].replace("\"\"", "\'").replace("\"", ""), x[2].replace("\"\"", "\'").replace("\"", "")), int(x[3]))))
similarQuestionAnswerScoreSqrt = similarQuestionAnswers.map(lambda x: (x[0], np.square(x[1][1]))).reduceByKey(add).map(lambda x:(x[0], np.sqrt(x[1])))
similarQuestionAnswerScoreSqrt = similarQuestionAnswerScoreSqrt.collectAsMap()

similarQuestionAnswerScoreNorm = similarQuestionAnswers.map(lambda x: (x[0], (x[1][0], x[1][1]/similarQuestionAnswerScoreSqrt.get(x[0]))))
similarQuestionAnswerScoreNorm = similarQuestionAnswerScoreNorm.map(lambda x: (x[0], (x[1][0], x[1][1] * similarQuestionJaccardSimilarity.get(x[0]))))
print(similarQuestionAnswerScoreNorm.take(1))

[Row(Id='185690', Body='"<p>I\'m using the <a href=""http://www.componentace.com/zlib_.NET.htm"" rel=""nofollow"">zlib.NET</a> library to try and inflate files that are compressed by zlib (on a Linux box; perhaps).  Here\'s what I\'m doing:</p>  <pre><code>zlib.ZInputStream zinput =     new zlib.ZInputStream(File.Open(path; FileMode.Open; FileAccess.Read));  while (stopByte != (data = zinput.ReadByte())) {   // check data here }  zinput.Close(); </code></pre>  <p>The data bytes match the compressed data bytes; so I must be doing something wrong.</p> "', Body="<p>It appears I made the mistake of assuming all virtual methods were overridden; which wasn't the case.  I was using zlib.ZInputStream.ReadByte(); which is just the inherited Stream.ReadByte(); which doesn't do any inflate.</p>  <p>I used zlib.ZInputStream.Read() instead; and it worked like it should.</p> ", Score='7')]
[('185690', (("<p>I'm using the <a href='http://www.componentace.com/zlib_.NET.htm' rel='nofollow'>zlib.NET</a>

In [28]:
answerCount = 10
similarQuestionAnswerSorted = similarQuestionAnswerScoreNorm.map(lambda x: x[1]).sortBy(lambda x: -x[1])
similarQuestionAnswerTop = similarQuestionAnswerSorted.take(answerCount)

In [29]:
with open('suggested_answers.csv', 'w') as suggestedAnswers:
        writer = csv.writer(suggestedAnswers, delimiter=',')
        writer.writerow(['Rank', 'Original Question', 'Answer'])
        for idx, questionAnswer in enumerate(similarQuestionAnswerTop):
            writer.writerow([idx + 1, questionAnswer[0][0], questionAnswer[0][1]])
            
for idx, questionAnswer in enumerate(similarQuestionAnswerTop):
    print("Rank: " + str(idx + 1))
    print("Original Question: " + questionAnswer[0][0])
    print("Answer: " + questionAnswer[0][1])
    print()

Rank: 1
Original Question: <p>I'm using the <a href='http://www.componentace.com/zlib_.NET.htm' rel='nofollow'>zlib.NET</a> library to try and inflate files that are compressed by zlib (on a Linux box; perhaps).  Here's what I'm doing:</p>  <pre><code>zlib.ZInputStream zinput =     new zlib.ZInputStream(File.Open(path; FileMode.Open; FileAccess.Read));  while (stopByte != (data = zinput.ReadByte())) {   // check data here }  zinput.Close(); </code></pre>  <p>The data bytes match the compressed data bytes; so I must be doing something wrong.</p> 
Answer: <p>It appears I made the mistake of assuming all virtual methods were overridden; which wasn't the case.  I was using zlib.ZInputStream.ReadByte(); which is just the inherited Stream.ReadByte(); which doesn't do any inflate.</p>  <p>I used zlib.ZInputStream.Read() instead; and it worked like it should.</p> 

Rank: 2
Original Question: <p>I'm using the <a href='http://www.componentace.com/zlib_.NET.htm' rel='nofollow'>zlib.NET</a> librar

In [ ]:
#count top 10 keyword:by tag and by word
#(word) title+body->upvotes
#(k-items) title+body->upvotes

In [ ]:
#count top 10 keyword by tag

In [ ]:
tagWords=tagLower.map(lambda x:(x[1],1))
tagCounts=tagWords.reduceByKey(add)
tagCounts.take(10)

In [ ]:
tagSorted = tagCounts.sortBy(lambda x: -x[1])
tagSorted.take(100)

In [ ]:
#count top 10 keyword by words
questionWords=questionPairStage1.map(lambda x:(x[0].split("@")[1],1))
questionCounts=questionWords.reduceByKey(add)
questionCounts.take(10)
print(questionCounts.take(100))

In [ ]:
questionSorted = questionCounts.sortBy(lambda x: -x[1])
questionSorted.take(100)

In [ ]:
#count top 10 keyword by keywords
def fmap2(x):
    outputs=[]
    for i in range(num_keywords):
        y=x[1][i].split("=")
        if float(y[1])>0.4:
           outputs.append((y[0],1))
    return (outputs)
keywordWords=questionPairStage4.flatMap(fmap2).reduceByKey(add)
keywordWords.take(10)

In [ ]:
keywordSorted = keywordWords.sortBy(lambda x: -x[1])
keywordSorted.take(100)

In [ ]:
print(1)

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.util import MLUtils
from pyspark.sql.functions import col
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.feature import IndexToString
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import HashingTF, IDF

In [ ]:
data = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('Questions.csv')

In [ ]:
dataSample = data.sample(False,0.10,80)
print(dataSample.take(2))

In [ ]:
drop_list = ['Id','Title', 'OwnerUserId', 'CreationDate', 'ClosedDate', 'Code']
dataInputs = dataSample.select([column for column in data.columns if column not in drop_list])
dataInputs.show(5)
dataInputs.printSchema()

In [ ]:
name = 'Score'
def labelfunction(x):
    if (int(x)>0):
        return "yes"
    else: return "no"
udf = UserDefinedFunction(labelfunction, StringType())
new_df = dataInputs.select(*[udf(column).alias(name) if column == name else column for column in dataInputs.columns])

In [ ]:
new_df = new_df.filter(new_df.Score.isNotNull())
#new_df = new_df.filter(new_df.Title.isNotNull())
new_df = new_df.filter(new_df.Body.isNotNull())
new_df.show(1000)

In [ ]:
#max_value = new_df.agg({"Score": "max"}).collect()[0][0]
#new_df=new_df.withColumn("Score", new_df["Score"].cast(IntegerType()))
#print(max_value)

In [ ]:
regexTokenizer = RegexTokenizer(inputCol="Body", outputCol="words", pattern="\\W")
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(stopWordsList)
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=1000000, minDF=50)
label_stringIdx = StringIndexer(inputCol = "Score", outputCol = "label")

In [ ]:

#label_stringIdx = IndexToString( inputCol="Score", outputCol="Label")
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])
#pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors])
pipelineFit = pipeline.fit(new_df)
dataset = pipelineFit.transform(new_df)
dataset.show(500)

In [ ]:
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 1)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))
trainingData.take(1)

In [ ]:
lr = LogisticRegression(maxIter=100, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.filter(predictions['prediction'] == 1) \
    .select("Body","Score","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 1000, truncate = 30)

In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

In [ ]:
nb = NaiveBayes(smoothing=1)
model = nb.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.filter(predictions['label'] == 1) \
    .select("Body","Score","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 1000, truncate = 30)

In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

In [ ]:
lr = LogisticRegression(maxIter=20, regParam=0.5, elasticNetParam=0.8)
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("Body","Score","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 200, truncate = 30)

In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

In [ ]:
rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)
rfModel = rf.fit(trainingData)
predictions = rfModel.transform(testData)
predictions.filter(predictions['prediction'] == 1) \
    .select("Body","Score","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 200, truncate = 30)

In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

In [ ]:
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=1000000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, hashingTF, idf, label_stringIdx])

In [ ]:
pipelineFit = pipeline.fit(new_df)
dataset = pipelineFit.transform(new_df)
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))
trainingData.take(1)

In [ ]:
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)
predictions = rfModel.transform(testData)
predictions.filter(predictions['prediction'] == 1) \
    .select("Body","Score","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 200, truncate = 30)

In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)